In [10]:
#!pip install fastapi==0.103.1 uvicorn==0.23.2 pydantic==2.3.0 python-multipart==0.0.6 jinja2==3.1.2 starlette==0.27.0 langchain-core==0.1.0 langchain-mistralai==0.0.3 langgraph==0.0.17 tavily-python==0.2.6 python-dotenv==1.0.0 requests==2.31.0 aiohttp==3.8.5

In [11]:
!pip install fastapi uvicorn pydantic jinja2 python-multipart
!pip install langchain-core langchain-mistralai langgraph
!pip install tavily-python
!pip install python-dotenv requests aiohttp

In [12]:
import os
import time
import json
import logging
import hashlib
import datetime
from typing import List, Dict, Any, Optional
from pathlib import Path
from pydantic import BaseModel, Field

# FastAPI imports
from fastapi import FastAPI, Request, HTTPException, Depends, Cookie
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
from starlette.responses import RedirectResponse

# Import research agent components
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, END

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('research_agent_api')

# Configuration class for the research agent
class AgentConfig:
    def __init__(
        self,
        tavily_api_key: str = None,
        mistral_api_key: str = None,
        mistral_model: str = "mistral-small",
        max_search_results: int = 5,
        temperature: float = 0.1,
        search_depth: str = "advanced",
        enable_cache: bool = True,
        cache_dir: str = "./research_cache",
        cache_ttl: int = 86400,  # 24 hours in seconds
        max_retries: int = 3,
        backoff_factor: float = 2.0,
        verbose: bool = True,
    ):
        self.tavily_api_key = tavily_api_key
        self.mistral_api_key = mistral_api_key
        self.mistral_model = mistral_model
        self.max_search_results = max_search_results
        self.temperature = temperature
        self.search_depth = search_depth
        self.enable_cache = enable_cache
        self.cache_dir = Path(cache_dir)
        self.cache_ttl = cache_ttl
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.verbose = verbose

        # Create cache directory if it doesn't exist
        if self.enable_cache:
            os.makedirs(self.cache_dir, exist_ok=True)

    @classmethod
    def from_env(cls, **kwargs):
        """Create a configuration from environment variables"""
        config = cls(**kwargs)

        # Override with environment variables if they exist
        if os.environ.get("TAVILY_API_KEY"):
            config.tavily_api_key = os.environ.get("TAVILY_API_KEY")
        if os.environ.get("MISTRAL_API_KEY"):
            config.mistral_api_key = os.environ.get("MISTRAL_API_KEY")
        if os.environ.get("MISTRAL_MODEL"):
            config.mistral_model = os.environ.get("MISTRAL_MODEL")

        return config

    def to_dict(self):
        """Convert config to dictionary for serialization"""
        return {
            "mistral_model": self.mistral_model,
            "max_search_results": self.max_search_results,
            "temperature": self.temperature,
            "search_depth": self.search_depth,
            "enable_cache": self.enable_cache,
            "cache_ttl": self.cache_ttl,
            "max_retries": self.max_retries,
            "verbose": self.verbose
        }

# Simple caching mechanism
class ResultCache:
    """Simple file-based cache for research results"""

    def __init__(self, config: AgentConfig):
        self.config = config
        self.cache_dir = config.cache_dir
        # Create cache directory if it doesn't exist
        os.makedirs(self.cache_dir, exist_ok=True)

    def _generate_key(self, query: str) -> str:
        """Generate a cache key from a query"""
        # Create a deterministic hash of the query
        return hashlib.md5(query.encode()).hexdigest()

    def _get_cache_path(self, key: str) -> Path:
        """Get the file path for a cache key"""
        return self.cache_dir / f"{key}.json"

    def get(self, query: str) -> Optional[Dict[str, Any]]:
        """Get cached results for a query if they exist and are fresh"""
        if not self.config.enable_cache:
            return None

        key = self._generate_key(query)
        cache_path = self._get_cache_path(key)

        if not cache_path.exists():
            return None

        try:
            # Check if cache is expired
            if time.time() - cache_path.stat().st_mtime > self.config.cache_ttl:
                logger.info(f"Cache expired for query: {query}")
                return None

            # Load cached data
            with open(cache_path, 'r') as f:
                data = json.load(f)

            logger.info(f"Cache hit for query: {query}")
            return data
        except Exception as e:
            logger.warning(f"Error reading cache: {str(e)}")
            return None

    def set(self, query: str, data: Dict[str, Any]) -> None:
        """Cache results for a query"""
        if not self.config.enable_cache:
            return

        key = self._generate_key(query)
        cache_path = self._get_cache_path(key)

        try:
            with open(cache_path, 'w') as f:
                json.dump(data, f)
            logger.info(f"Cached results for query: {query}")
        except Exception as e:
            logger.warning(f"Error writing cache: {str(e)}")

# Create a direct Tavily search function with caching
def create_search_function(config: AgentConfig):
    """Create a function that searches using Tavily API directly with caching"""
    from tavily import TavilyClient

    # Initialize cache
    cache = ResultCache(config)

    # Explicitly pass the API key to the client
    tavily_client = TavilyClient(api_key=config.tavily_api_key)

    def search(query: str) -> List[Dict[str, Any]]:
        """Search the web using Tavily API with caching and retries"""
        try:
            # Check cache first
            cached_results = cache.get(query)
            if cached_results:
                logger.info(f"Using cached results for: {query}")
                return cached_results

            logger.info(f"Searching for: {query}")

            # Try to search with retries and exponential backoff
            max_retries = config.max_retries
            backoff = 1

            for attempt in range(max_retries):
                try:
                    response = tavily_client.search(
                        query=query,
                        search_depth=config.search_depth,
                        max_results=config.max_search_results
                    )
                    results = response.get("results", [])

                    # Cache the results
                    cache.set(query, results)

                    return results

                except Exception as e:
                    if attempt < max_retries - 1:
                        wait_time = backoff * config.backoff_factor
                        logger.warning(f"Search error, retrying in {wait_time}s: {str(e)}")
                        time.sleep(wait_time)
                        backoff *= config.backoff_factor
                    else:
                        raise

            # Fallback to basic search if we get here
            logger.info("Falling back to basic search...")

            response = tavily_client.search(
                query=query,
                search_depth="basic",
                max_results=config.max_search_results
            )
            results = response.get("results", [])

            # Cache the results
            cache.set(query, results)

            return results

        except Exception as e:
            logger.error(f"Search failed after all retry attempts: {str(e)}")
            return []

    return search

# Initialize the LLM
def create_llm(config: AgentConfig):
    """Create and configure the LLM"""
    try:
        llm = ChatMistralAI(
            model=config.mistral_model,
            temperature=config.temperature,
            max_tokens=4000,  # Allow for longer responses
            api_key=config.mistral_api_key  # Explicitly pass the API key
        )
        return llm
    except Exception as e:
        logger.error(f"Error creating LLM: {str(e)}")
        raise

# Define the state schema for our agent system
class AgentState(dict):
    """Dictionary subclass for agent state with typed annotations for clarity"""
    query: str
    refined_query: Optional[str]
    research_results: List[Dict[str, Any]]
    research_summary: str
    draft_answer: str
    final_answer: str
    messages: List[Any]  # List of message objects
    start_time: float
    metadata: Dict[str, Any]
    error_count: int

# Query refinement node
def refine_query(state: AgentState, config: AgentConfig) -> AgentState:
    """Refine the user's query to make it more effective for research"""
    logger.info("Refining query: Optimizing for better search results")

    try:
        # Extract query from state
        query = state["query"]

        # Create LLM
        llm = create_llm(config)

        # Create messages for query refinement
        messages = [
            SystemMessage(content="""You are a query optimization expert. Your task is to refine search queries to get better search results.
            Make the query more specific, clear, and effective for web searching. Do not change the original intent of the query.
            If the query is already well-formulated, you can return it as is."""),
            HumanMessage(content=f"""
            Original Query: {query}

            Please refine this query to make it more effective for web searching. Focus on:
            1. Making it more specific if needed
            2. Adding relevant keywords
            3. Removing unnecessary words
            4. Making it clearer and more precise

            Return ONLY the refined query, with no additional explanation or commentary.
            """)
        ]

        # Get refined query
        logger.info("Generating refined query...")

        refined_response = llm.invoke(messages)
        refined_query = refined_response.content.strip()

        # Check if query was actually refined
        if refined_query.lower() == query.lower():
            refined_query = query
            logger.info("Query is already well-formulated")
        else:
            logger.info(f"Query refined: {refined_query}")

        # Update state
        state["refined_query"] = refined_query

    except Exception as e:
        logger.error(f"Error in query refinement: {str(e)}")
        # Fall back to original query
        state["refined_query"] = state["query"]
        # Increment error count
        state["error_count"] += 1

    return state

# Research node implementation with progress tracking
def research(state: AgentState, config: AgentConfig) -> AgentState:
    """Research node: Search for information and summarize findings"""
    # Extract query from state
    query = state["query"]
    refined_query = state.get("refined_query", query)
    search_query = refined_query if refined_query else query

    logger.info(f"RESEARCH: Searching for information on '{search_query}'")

    try:
        # Get search function and LLM
        search_function = create_search_function(config)
        llm = create_llm(config)

        # Execute search
        search_results = search_function(search_query)

        if not search_results:
            logger.warning("No search results found")
            research_summary = "The search did not return any results. Please try a different query or check your internet connection."
            state["research_results"] = []
            state["research_summary"] = research_summary
            state["messages"] = [
                HumanMessage(content=query),
                AIMessage(content=research_summary)
            ]
            state["error_count"] += 1
            return state

        # Format search results for the LLM
        results_text = ""
        for i, result in enumerate(search_results):
            results_text += f"Source {i+1}: {result.get('title', 'No title')}\n"
            results_text += f"URL: {result.get('url', 'No URL')}\n"
            results_text += f"Content: {result.get('content', 'No content')}\n\n"

        # Create messages for the LLM
        messages = [
            SystemMessage(content="""You are a research assistant. Based on the search results provided, create a comprehensive summary.
            Focus on synthesizing information from multiple sources, identifying key facts, insights, and different perspectives.
            Be objective and thorough in your summary. Include relevant statistics, dates, and findings from the search results.
            Identify any conflicting information or gaps in the available data."""),
            HumanMessage(content=f"""
            Search Query: {search_query}

            Search Results:
            {results_text}

            Please provide a detailed summary of the information found in these search results.
            Focus on key facts, insights, and different perspectives on the topic.
            Highlight any consensus views as well as areas of debate or uncertainty.
            """)
        ]

        # Get research summary
        logger.info("Generating research summary...")
        summary_response = llm.invoke(messages)
        research_summary = summary_response.content

        logger.info("Research summary generated")

        # Update state
        state["research_results"] = search_results
        state["research_summary"] = research_summary
        state["messages"] = [
            HumanMessage(content=query),
            summary_response
        ]

    except Exception as e:
        logger.error(f"Error in research node: {str(e)}")
        # Provide graceful failure
        state["research_results"] = []
        state["research_summary"] = f"Research failed due to an error: {str(e)}. Please try again later."
        state["messages"] = [
            HumanMessage(content=query),
            AIMessage(content=f"I encountered an error while researching: {str(e)}. Please try again or modify your query.")
        ]
        # Increment error count
        state["error_count"] += 1

    return state

# Draft answer node implementation
def draft_answer(state: AgentState, config: AgentConfig) -> AgentState:
    """Draft answer node: Create initial answer based on research"""
    logger.info("DRAFTING: Creating initial answer based on research")

    try:
        # Extract required information
        query = state["query"]
        research_summary = state["research_summary"]

        # Handle case where research failed
        if not research_summary or research_summary.startswith("Research failed"):
            logger.warning("Research summary not available, creating limited draft")
            state["draft_answer"] = "Unable to provide a comprehensive answer due to research limitations."
            state["error_count"] += 1
            return state

        # Get LLM
        llm = create_llm(config)

        # Create messages for drafting
        messages = [
            SystemMessage(content="""You are an expert content writer. Your task is to draft a comprehensive answer based on the research summary provided.
            Focus on accuracy, clarity, and logical organization of information. Present different perspectives where appropriate.
            Use paragraphs, bullet points, and other formatting as needed to make the content readable and engaging.
            Cite specific facts, statistics, and findings from the research summary. Don't add information not found in the research."""),
            HumanMessage(content=f"""
            Original Query: {query}

            Research Summary:
            {research_summary}

            Create a well-structured draft answer based on this research. Focus on clarity, accuracy, and coherence.
            Organize the information logically and make it engaging. Include all relevant facts and insights from the research.
            """)
        ]

        # Get draft answer
        logger.info("Generating draft answer...")
        draft_response = llm.invoke(messages)
        draft_answer = draft_response.content

        logger.info("Draft answer generated")

        # Update state
        state["draft_answer"] = draft_answer

    except Exception as e:
        logger.error(f"Error in draft node: {str(e)}")
        # Provide graceful failure
        state["draft_answer"] = f"Unable to draft an answer due to an error: {str(e)}. Please try again later."
        # Increment error count
        state["error_count"] += 1

    return state

# Finalize answer node implementation
def finalize_answer(state: AgentState, config: AgentConfig) -> AgentState:
    """Finalize answer node: Polish and improve the draft"""
    logger.info("FINALIZING: Polishing and improving the answer")

    try:
        # Extract required information
        query = state["query"]
        research_summary = state["research_summary"]
        draft_answer = state["draft_answer"]

        # Handle case where drafting failed
        if not draft_answer or draft_answer.startswith("Unable to draft"):
            logger.warning("Draft not available, providing limited final answer")
            state["final_answer"] = "I couldn't complete the research process successfully. Please try again with a different query."
            state["error_count"] += 1
            return state

        # Get LLM
        llm = create_llm(config)

        # Create messages for finalizing
        messages = [
            SystemMessage(content="""You are an expert editor and reviewer. Your task is to review the draft answer and improve it into a final version.
            Focus on enhancing clarity, readability, and coherence while maintaining accuracy.
            Improve the structure, flow, and transitions between sections.
            Ensure consistent tone, eliminate redundancies, and correct any grammatical or stylistic issues.
            Make the answer more engaging and accessible without compromising on substance."""),
            HumanMessage(content=f"""
            Original Query: {query}

            Research Summary:
            {research_summary}

            Draft Answer:
            {draft_answer}

            Please review the draft and provide a polished final answer. Ensure it is:
            1. Accurate - correctly represents the research findings
            2. Comprehensive - includes all important information
            3. Well-structured - follows a logical flow
            4. Clear and engaging - easy to understand
            5. Free of errors - grammatically correct and properly formatted

            Provide only the final answer without any meta-commentary.
            """)
        ]

        # Get final answer
        logger.info("Generating final answer...")
        final_response = llm.invoke(messages)
        final_answer = final_response.content

        logger.info("Final answer generated")

        # Update state
        state["final_answer"] = final_answer

        # Update metadata with timing information
        elapsed_time = time.time() - state["start_time"]
        state["metadata"]["total_time"] = elapsed_time

    except Exception as e:
        logger.error(f"Error in finalize node: {str(e)}")
        # Provide graceful failure
        state["final_answer"] = f"I was unable to finalize the answer due to an error: {str(e)}. Here is the draft answer instead:\n\n{state['draft_answer']}"
        # Increment error count
        state["error_count"] += 1

    return state

# Create the graph with configuration
def create_research_workflow(config: AgentConfig):
    """Create and compile the agent workflow graph"""
    logger.info("Creating research workflow graph...")

    try:
        # Initialize the graph
        workflow = StateGraph(AgentState)

        # Add nodes with configuration as argument
        workflow.add_node("refine_query", lambda state: refine_query(state, config))
        workflow.add_node("research", lambda state: research(state, config))
        workflow.add_node("draft", lambda state: draft_answer(state, config))
        workflow.add_node("finalize", lambda state: finalize_answer(state, config))

        # Add edges with the new query refinement step
        workflow.add_edge("refine_query", "research")
        workflow.add_edge("research", "draft")
        workflow.add_edge("draft", "finalize")
        workflow.add_edge("finalize", END)

        # Set the entry point
        workflow.set_entry_point("refine_query")

        # Compile the graph
        compiled_workflow = workflow.compile()

        logger.info("Workflow graph created and compiled successfully")

        return compiled_workflow

    except Exception as e:
        logger.error(f"Error creating workflow graph: {str(e)}")
        raise

# Function to run the research system with improved error handling
def run_research_system(query: str, config: AgentConfig):
    """Run the full research workflow"""
    start_time = time.time()

    logger.info(f"STARTING RESEARCH WORKFLOW: '{query}'")

    try:
        # Input validation
        if not query or not query.strip():
            logger.error("Query cannot be empty")
            return {
                "query": "",
                "refined_query": "",
                "research_results": [],
                "research_summary": "No query provided",
                "draft_answer": "",
                "final_answer": "Please provide a valid search query",
                "messages": [],
                "start_time": start_time,
                "metadata": {"error": "Empty query"},
                "error_count": 1
            }

        # Create the workflow with configuration
        research_workflow = create_research_workflow(config)

        # Initialize the state
        initial_state = {
            "query": query,
            "refined_query": None,
            "research_results": [],
            "research_summary": "",
            "draft_answer": "",
            "final_answer": "",
            "messages": [],
            "start_time": start_time,
            "metadata": {"config": config.to_dict()},
            "error_count": 0
        }

        # Execute the workflow
        result = research_workflow.invoke(initial_state)

        # Calculate execution time
        execution_time = time.time() - start_time

        logger.info(f"Research completed in {execution_time:.2f} seconds!")

        return result

    except Exception as e:
        logger.error(f"Error in research workflow: {str(e)}")
        # Return partial results
        return {
            "query": query,
            "refined_query": None,
            "research_results": [],
            "research_summary": f"Research workflow failed: {str(e)}",
            "draft_answer": "",
            "final_answer": f"I encountered an error while processing your query: {str(e)}. Please try again later.",
            "messages": [],
            "start_time": start_time,
            "metadata": {"error": str(e)},
            "error_count": 1
        }

# Pydantic models for API requests and responses
class SetupRequest(BaseModel):
    tavily_api_key: str
    mistral_api_key: str
    model: str = "mistral-small"
    max_results: int = 5
    temperature: float = 0.1
    search_depth: str = "advanced"
    enable_cache: bool = True

class ResearchRequest(BaseModel):
    query: str
    model: Optional[str] = None
    max_results: Optional[int] = None
    temperature: Optional[float] = None
    search_depth: Optional[str] = None
    enable_cache: Optional[bool] = None

class ResearchResponse(BaseModel):
    query: str
    refined_query: str = ""
    research_summary: str = ""
    draft_answer: str = ""
    final_answer: str = ""
    research_results: List[Dict[str, Any]] = []
    timestamp: str = ""
    metadata: Dict[str, Any] = {}

class ConfigResponse(BaseModel):
    mistral_model: str
    max_search_results: int
    temperature: float
    search_depth: str
    enable_cache: bool
    cache_ttl: int = 86400
    max_retries: int = 3
    verbose: bool = True

# Create FastAPI app
app = FastAPI(title="Research Agent API")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Create a simple in-memory store for session data
# In a production app, you'd use a database or Redis
session_store = {}

# Create templates directory
templates = Jinja2Templates(directory="templates")

# Create templates directory
os.makedirs("templates", exist_ok=True)

# Write the index.html template
with open("templates/index.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Research Agent</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.10.0/font/bootstrap-icons.css">
    <style>
        body {
            padding-top: 20px;
            background-color: #f8f9fa;
        }
        .research-card {
            transition: all 0.3s;
        }
        .research-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 20px rgba(0,0,0,0.1);
        }
        .loading {
            display: none;
        }
        .api-key-section {
            background-color: #f0f8ff;
            border-radius: 10px;
            padding: 20px;
            margin-bottom: 20px;
        }
        .history-item {
            cursor: pointer;
        }
        .history-item:hover {
            background-color: #f0f8ff;
        }
        #searchResults {
            max-height: 300px;
            overflow-y: auto;
        }
        .source-url {
            word-break: break-all;
        }
        .tab-content {
            padding: 20px;
            background-color: #fff;
            border: 1px solid #dee2e6;
            border-top: none;
            border-radius: 0 0 5px 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1 class="text-center mb-4">Research Assistant</h1>

        <!-- API Key Setup -->
        <div class="api-key-section" id="apiKeySection">
            <h3>Setup API Keys</h3>
            <div class="mb-3">
                <label for="tavilyApiKey" class="form-label">Tavily API Key</label>
                <input type="password" class="form-control" id="tavilyApiKey" placeholder="Enter Tavily API Key">
            </div>
            <div class="mb-3">
                <label for="mistralApiKey" class="form-label">Mistral API Key</label>
                <input type="password" class="form-control" id="mistralApiKey" placeholder="Enter Mistral API Key">
            </div>
            <div class="mb-3">
                <label for="modelSelect" class="form-label">Model</label>
                <select class="form-select" id="modelSelect">
                    <option value="mistral-small">Mistral Small</option>
                    <option value="mistral-medium">Mistral Medium</option>
                    <option value="mistral-large">Mistral Large</option>
                </select>
            </div>
            <div class="row">
                <div class="col-md-6">
                    <div class="mb-3">
                        <label for="maxResults" class="form-label">Max Search Results</label>
                        <input type="number" class="form-control" id="maxResults" value="5" min="1" max="10">
                    </div>
                </div>
                <div class="col-md-6">
                    <div class="mb-3">
                        <label for="temperature" class="form-label">Temperature</label>
                        <input type="range" class="form-range" id="temperature" min="0" max="1" step="0.1" value="0.1">
                        <span id="temperatureValue">0.1</span>
                    </div>
                </div>
            </div>
            <div class="mb-3">
                <label for="searchDepth" class="form-label">Search Depth</label>
                <select class="form-select" id="searchDepth">
                    <option value="basic">Basic</option>
                    <option value="advanced" selected>Advanced</option>
                </select>
            </div>
            <div class="mb-3 form-check">
                <input type="checkbox" class="form-check-input" id="enableCache" checked>
                <label class="form-check-label" for="enableCache">Enable Caching</label>
            </div>
            <button class="btn btn-primary" id="setupBtn">Save Settings</button>
        </div>

        <!-- Research Form -->
        <div class="card mb-4">
            <div class="card-body">
                <form id="researchForm">
                    <div class="mb-3">
                        <label for="query" class="form-label">What would you like to research?</label>
                        <input type="text" class="form-control" id="query" placeholder="Enter your research query">
                    </div>
                    <button type="submit" class="btn btn-success" id="researchBtn">
                        <i class="bi bi-search"></i> Research
                    </button>
                    <div class="spinner-border text-primary loading" id="loading" role="status">
                        <span class="visually-hidden">Loading...</span>
                    </div>
                </form>
            </div>
        </div>

        <!-- Results Section -->
        <div class="card mb-4" id="resultsCard" style="display: none;">
            <div class="card-header">
                <ul class="nav nav-tabs card-header-tabs" id="resultTabs">
                    <li class="nav-item">
                        <a class="nav-link active" data-bs-toggle="tab" href="#answerTab">Answer</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" data-bs-toggle="tab" href="#summaryTab">Research Summary</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" data-bs-toggle="tab" href="#sourcesTab">Sources</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" data-bs-toggle="tab" href="#processTab">Process</a>
                    </li>
                </ul>
            </div>
            <div class="card-body">
                <div class="tab-content">
                    <div class="tab-pane fade show active" id="answerTab">
                        <h4 id="queryDisplay"></h4>
                        <div id="finalAnswer" class="mt-3"></div>
                    </div>
                    <div class="tab-pane fade" id="summaryTab">
                        <div id="researchSummary"></div>
                    </div>
                    <div class="tab-pane fade" id="sourcesTab">
                        <div id="searchResults" class="list-group"></div>
                    </div>
                    <div class="tab-pane fade" id="processTab">
                        <div class="mb-3">
                            <h5>Original Query</h5>
                            <p id="originalQuery"></p>
                        </div>
                        <div class="mb-3">
                            <h5>Refined Query</h5>
                            <p id="refinedQuery"></p>
                        </div>
                        <div class="mb-3">
                            <h5>Draft Answer</h5>
                            <div id="draftAnswer"></div>
                        </div>
                        <div class="mb-3">
                            <h5>Metadata</h5>
                            <pre id="metadata" class="bg-light p-3 rounded"></pre>
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <!-- Search History -->
        <div class="card" id="historyCard">
            <div class="card-header bg-light">
                <h5>Search History</h5>
            </div>
            <div class="card-body">
                <ul class="list-group" id="historyList">
                    <!-- History items will be added here -->
                </ul>
                <div class="text-center mt-3" id="noHistory">
                    <p class="text-muted">No search history yet</p>
                </div>
            </div>
        </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/js/bootstrap.bundle.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <script>
        // Initialize variables
        let isConfigured = false;
        let searchHistory = [];

        // DOM Elements
        const setupBtn = document.getElementById('setupBtn');
        const researchForm = document.getElementById('researchForm');
        const resultsCard = document.getElementById('resultsCard');
        const apiKeySection = document.getElementById('apiKeySection');
        const historyList = document.getElementById('historyList');
        const noHistory = document.getElementById('noHistory');

        // Check if API keys are already set
        async function checkConfiguration() {
            try {
                const response = await fetch('/api/config');
                if (response.ok) {
                    const config = await response.json();
                    isConfigured = true;
                    updateConfigUI(config);
                    loadSearchHistory();
                }
            } catch (error) {
                console.error('Error checking configuration:', error);
            }
        }

        // Update the configuration UI
        function updateConfigUI(config) {
            document.getElementById('modelSelect').value = config.mistral_model;
            document.getElementById('maxResults').value = config.max_search_results;
            document.getElementById('temperature').value = config.temperature;
            document.getElementById('temperatureValue').textContent = config.temperature;
            document.getElementById('searchDepth').value = config.search_depth;
            document.getElementById('enableCache').checked = config.enable_cache;
        }

        // Load search history
        async function loadSearchHistory() {
            try {
                const response = await fetch('/api/history');
                if (response.ok) {
                    searchHistory = await response.json();
                    renderSearchHistory();
                }
            } catch (error) {
                console.error('Error loading search history:', error);
            }
        }

        // Render search history
        function renderSearchHistory() {
            historyList.innerHTML = '';

            if (searchHistory.length === 0) {
                noHistory.style.display = 'block';
                return;
            }

            noHistory.style.display = 'none';

            searchHistory.forEach((item, index) => {
                const li = document.createElement('li');
                li.className = 'list-group-item history-item';
                li.innerHTML = `
                    <div class="d-flex justify-content-between align-items-center">
                        <span>${item.query}</span>
                        <span class="text-muted small">${new Date(item.timestamp).toLocaleString()}</span>
                    </div>
                `;
                li.addEventListener('click', () => loadHistoryItem(index));
                historyList.appendChild(li);
            });
        }

        // Load a history item
        function loadHistoryItem(index) {
            const item = searchHistory[index];
            displayResults(item);
        }

        // Setup API keys
        setupBtn.addEventListener('click', async () => {
            const tavilyApiKey = document.getElementById('tavilyApiKey').value;
            const mistralApiKey = document.getElementById('mistralApiKey').value;
            const model = document.getElementById('modelSelect').value;
            const maxResults = parseInt(document.getElementById('maxResults').value);
            const temperature = parseFloat(document.getElementById('temperature').value);
            const searchDepth = document.getElementById('searchDepth').value;
            const enableCache = document.getElementById('enableCache').checked;

            if (!tavilyApiKey || !mistralApiKey) {
                alert('Please enter both API keys');
                return;
            }

            try {
                const response = await fetch('/api/setup', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        tavily_api_key: tavilyApiKey,
                        mistral_api_key: mistralApiKey,
                        model,
                        max_results: maxResults,
                        temperature,
                        search_depth: searchDepth,
                        enable_cache: enableCache
                    })
                });

                if (response.ok) {
                    isConfigured = true;
                    alert('Configuration saved successfully!');
                } else {
                    const error = await response.json();
                    alert(`Error: ${error.detail || 'Failed to save configuration'}`);
                }
            } catch (error) {
                console.error('Error setting up API keys:', error);
                alert('Failed to save configuration. Please try again.');
            }
        });

        // Update temperature value display
        document.getElementById('temperature').addEventListener('input', (e) => {
            document.getElementById('temperatureValue').textContent = e.target.value;
        });

        // Submit research form
        researchForm.addEventListener('submit', async (e) => {
            e.preventDefault();

            if (!isConfigured) {
                alert('Please set up API keys first');
                return;
            }

            const query = document.getElementById('query').value.trim();
            if (!query) {
                alert('Please enter a research query');
                return;
            }

            document.getElementById('researchBtn').style.display = 'none';
            document.getElementById('loading').style.display = 'inline-block';

            try {
                const response = await fetch('/api/research', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        query,
                        model: document.getElementById('modelSelect').value,
                        max_results: parseInt(document.getElementById('maxResults').value),
                        temperature: parseFloat(document.getElementById('temperature').value),
                        search_depth: document.getElementById('searchDepth').value,
                        enable_cache: document.getElementById('enableCache').checked
                    })
                });

                if (response.ok) {
                    const result = await response.json();
                    displayResults(result);
                    // Add to history if not already there
                    if (!searchHistory.some(item => item.query === result.query)) {
                        searchHistory.unshift(result);
                        renderSearchHistory();
                    }
                } else {
                    const error = await response.json();
                    alert(`Error: ${error.detail || 'Research failed'}`);
                }
            } catch (error) {
                console.error('Error performing research:', error);
                alert('Research failed. Please try again.');
            } finally {
                document.getElementById('researchBtn').style.display = 'inline-block';
                document.getElementById('loading').style.display = 'none';
            }
        });

        // Display research results
        function displayResults(result) {
            // Show results card
            resultsCard.style.display = 'block';

            // Display query
            document.getElementById('queryDisplay').textContent = result.query;
            document.getElementById('originalQuery').textContent = result.query;
            document.getElementById('refinedQuery').textContent = result.refined_query || 'No refinement needed';

            // Display answers using Markdown
            document.getElementById('finalAnswer').innerHTML = marked.parse(result.final_answer);
            document.getElementById('draftAnswer').innerHTML = marked.parse(result.draft_answer);
            document.getElementById('researchSummary').innerHTML = marked.parse(result.research_summary);

            // Display metadata
            document.getElementById('metadata').textContent = JSON.stringify(result.metadata, null, 2);

            // Display search results
            const searchResultsContainer = document.getElementById('searchResults');
            searchResultsContainer.innerHTML = '';

            if (result.research_results && result.research_results.length > 0) {
                result.research_results.forEach((source, idx) => {
                    const item = document.createElement('div');
                    item.className = 'list-group-item';
                    item.innerHTML = `
                        <h5>${source.title || 'No title'}</h5>
                        <p class="source-url">
                            <a href="${source.url}" target="_blank">${source.url}</a>
                        </p>
                        <p>${source.content ? source.content.substring(0, 200) + '...' : 'No content'}</p>
                    `;
                    searchResultsContainer.appendChild(item);
                });
            } else {
                searchResultsContainer.innerHTML = '<div class="list-group-item">No sources found</div>';
            }

            // Scroll to results
            resultsCard.scrollIntoView({ behavior: 'smooth' });
        }

        // Check configuration on page load
        checkConfiguration();
    </script>
</body>
</html>
""")

# Create static directory
os.makedirs("static", exist_ok=True)

# Mount static files directory
app.mount("/static", StaticFiles(directory="static"), name="static")

# Global configuration
global_config = None

# Get browser session
def get_session(session_id: str = Cookie(None)):
    if not session_id:
        session_id = f"session_{hashlib.md5(str(time.time()).encode()).hexdigest()}"
    if session_id not in session_store:
        session_store[session_id] = {}
    return session_id, session_store[session_id]

# Setup endpoints
@app.get("/", response_class=HTMLResponse)
async def get_home(request: Request):
    logger.info("Loading home page")
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/api/setup")
async def setup(request: SetupRequest):
    global global_config

    try:
        # Create configuration with API keys
        config = AgentConfig(
            tavily_api_key=request.tavily_api_key,
            mistral_api_key=request.mistral_api_key,
            mistral_model=request.model,
            max_search_results=request.max_results,
            temperature=request.temperature,
            search_depth=request.search_depth,
            enable_cache=request.enable_cache
        )

        # Store the configuration globally
        global_config = config

        logger.info("API keys and configuration saved successfully")

        return {"status": "success", "message": "Configuration saved successfully"}

    except Exception as e:
        logger.error(f"Error saving API keys: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Failed to save configuration: {str(e)}")

@app.get("/api/config", response_model=ConfigResponse)
async def get_config():
    if not global_config:
        raise HTTPException(status_code=404, detail="Configuration not found")

    return {
        "mistral_model": global_config.mistral_model,
        "max_search_results": global_config.max_search_results,
        "temperature": global_config.temperature,
        "search_depth": global_config.search_depth,
        "enable_cache": global_config.enable_cache,
        "cache_ttl": global_config.cache_ttl,
        "max_retries": global_config.max_retries,
        "verbose": global_config.verbose
    }

# Search history storage - in production, use a database instead
search_history = []

@app.post("/api/research", response_model=ResearchResponse)
async def research_query(request: ResearchRequest):
    global global_config

    if not global_config:
        raise HTTPException(status_code=400, detail="API keys not configured. Please setup first.")

    try:
        # Create a copy of the global config
        config = AgentConfig(
            tavily_api_key=global_config.tavily_api_key,
            mistral_api_key=global_config.mistral_api_key,
            mistral_model=request.model or global_config.mistral_model,
            max_search_results=request.max_results or global_config.max_search_results,
            temperature=request.temperature or global_config.temperature,
            search_depth=request.search_depth or global_config.search_depth,
            enable_cache=request.enable_cache if request.enable_cache is not None else global_config.enable_cache
        )

        # Run the research
        result = run_research_system(request.query, config)

        # Format the response
        response = {
            "query": result["query"],
            "refined_query": result.get("refined_query", ""),
            "research_summary": result["research_summary"],
            "draft_answer": result["draft_answer"],
            "final_answer": result["final_answer"],
            "research_results": result.get("research_results", []),
            "timestamp": datetime.datetime.now().isoformat(),
            "metadata": result.get("metadata", {})
        }

        # Add to history
        if len(search_history) >= 50:  # Limit history size
            search_history.pop()
        search_history.insert(0, response)

        return response

    except Exception as e:
        logger.error(f"Research error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Research failed: {str(e)}")

@app.get("/api/history")
async def get_history():
    return search_history

# Run the app using uvicorn# Modify the main block at the end of your script
if __name__ == "__main__":
    # Load API keys from environment if available
    config = AgentConfig.from_env()
    global_config = config

    # Don't call uvicorn directly in Colab
    logger.info("FastAPI app is configured and ready")
    print("To run this app, use the ngrok method below instead of the direct uvicorn call")

To run this app, use the ngrok method below instead of the direct uvicorn call


In [13]:
# Import necessary libraries
import nest_asyncio
import uvicorn
import threading
from google.colab import output

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Run the FastAPI app in a separate thread
def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Start the app in a thread
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Use Colab's port forwarding to expose the app
output.serve_kernel_port_as_window(8000)

print("FastAPI app is running. Click the URL above to access it.")

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

FastAPI app is running. Click the URL above to access it.


INFO:     Started server process [587]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
